# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [2]:
# !pip install matplotlib
# !pip install  fastparquet==0.7.1
# !pip install geopandas
# !pip install shapely
# !pip install geopy

In [4]:
# all import statements needed for the project, for example:
import math
import requests
import re
import bs4
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import geopandas as gpd
import geopy.distance
import shapely
import os.path

In [5]:
# any general notebook setup, like log formatting


In [1]:
# any constants you might need, for example:

TAXI_URL = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"
weather_2009_CSV="2009_weather.csv"
weather_2010_CSV="2010_weather.csv"
weather_2011_CSV="2011_weather.csv"
weather_2012_CSV="2012_weather.csv"
weather_2013_CSV="2013_weather.csv"
weather_2014_CSV="2014_weather.csv"


NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
For uber dataset, since there is no "distance" column, we need to calculate distance based on pickup's and dropoff's longitude and latitude. We use the euclidean distance between start and end coordinate. In order to keep datasets consistant, we ignore the "distance" column in yellow taxi dataset and calculate new "distance" column (using the same process as uber dataset does)

The function calculate_distance(picklon, picklat,droplon,droplat) will take pickup's and dropoff's longitudes and lantitudes, and then return the euclidean distance.

In [16]:
def calculate_distance(picklon, picklat,droplon,droplat):
    #return the euclidean distance between start and end coordinate
    from math import sin, cos, sqrt, atan2, radians
    R = 6373.0 #approximate radius of earth in km
    lon1=picklon
    lat1=picklat
    lon2=droplon
    lat2=droplat
    dlon=lon2-lon1
    dlat=lat2-lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    b = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance=R*b
    return distance

Function get_lon_lat(dataframe) is used to deal with taxi datasets, since some taxi datasets only contain location ID. We need to convert ID to latitude and longitude.get_lon_lat(dataframe) will return dataframe with four new columns "pickup_longitude","pickup_latitude","dropoff_longitude",and "dropoff_latitude".

In [ ]:
def get_lon_lat(dataframe):
    zone_file=gpd.read_file("taxi_zones")
    zone_file['geometry']=zone_file['geometry'].to_crs(4326)
    zone_file["lon"]=zone_file['geometry'].centroid.x
    zone_file["lat"]=zone_file['geometry'].centroid.y
    pick=zone_file[["LocationID","lon","lat"]]
    #rename columns
    pick.columns=["PULocationID","pickup_longitude","pickup_latitude"]
    drop=zone_file[["LocationID","lon","lat"]]
    #rename columns
    drop.columns=["DOLocationID","dropoff_longitude","dropoff_latitude"]
    #merge pick and drop to dataframe
    #merge pick 
    dataframe=pd.merge(left=dataframe,right=pick)
    #merge drop
    dataframe=pd.merge(left=dataframe,right=drop)
    return dataframe

Function add_distance_column(dataframe) will add new column "distance" through applying function calculate_distance to each row of the dataframe, which using columns "pickup_longitude","pickup_latitude","dropoff_longitude",and "dropoff_latitude".

In [17]:
def add_distance_column(dataframe):
    dataframe["distance"]=dataframe.apply(lambda x: calculate_distance(x["pickup_longitude"],x["pickup_latitude"],
                                                             x["dropoff_longitude"],x["dropoff_latitude"]),axis=1)
    return dataframe 

### Processing Taxi Data


After exploring yellow taxi datasets, we find that dataset's columns, column names, datatype are different among different year. Thus, we need to clean taxi data for following analysis.

First, we write a function called find_taxi_Parquet_urls, which will return urls for Parquet dataset from January 2009 through June 2015.

In [24]:
def find_taxi_Parquet_urls():
    url=TAXI_URL #url for taxi data page is set above
    url_list=list()
    response=requests.get(url)
    if response.status_code!=200:
        print("Failure: fail to request data")
        return result
    try:
        result_page=BeautifulSoup(response.content,'lxml')
        result_tags=result_page.find_all("a",{"title":"Yellow Taxi Trip Records"})
        for element in result_tags:
            newurl=element.get('href')
            url_list.append(newurl)
        #only include dataset link from January 2009 through June 2015 
        result=url_list[80:86]
        result.extend(url_list[92:]) 
    except:
        print("Failure: scraping fail")
    return result



In [32]:
#example of using find_taxi_Parquet_urls()
result=find_taxi_Parquet_urls()
result[0]

'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-06.parquet'

Function get_and_clean_month_taxi_data(url) will take one url for taxi parque as input. Then, it will normalize column name, normalize data type, drop null value, remove data point outside the coordinate box, and sampling the data based on uber dataset's size. The output of the function will be a cleaned dataframe

In [28]:
#clean data
def get_and_clean_month_taxi_data(url):
    #get month Taxi data
    df=pd.read_parquet(url,engine='fastparquet')
    
    #possible column name after looking at each dataset
     #list1=['tpep_pickup_datetime',"trip_distance","PULocationID","DOLocationID","tip_amount"]
     #list2=['pickup_datetime','trip_distance','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','tip_amount']
     #list3=['Trip_Pickup_DateTime','Trip_Distance','Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Tip_Amt']
    
    #deal with df.column contains list1
    if df.columns.str.contains('tpep_pickup_datetime').any():#df.column contain list1         
        #select column
        df=df[["tpep_pickup_datetime","PULocationID","DOLocationID","tip_amount"]]
        #From taxi_zone, we know that loction ID is valid between 1 and 263. Thus, we can use it to clean our taxi dataset.
        #remove invalid locationID：location ID should between 1 and 263, and should be numeric
        df=df.loc[(df["PULocationID"]>=1) & (df["PULocationID"]<=263) & (df["DOLocationID"]>=1)
                & (df["DOLocationID"]<=263)]
       
        #only select pickup id not equal to drop off id
        df=df.loc[df["PULocationID"]!=df["DOLocationID"]]
        #add longitude and latitude based on location ID
        df=get_lon_lat(df)
        #delete id
        del df["DOLocationID"]
        del df["PULocationID"]
        #rename 'tpep_pickup_datetime'
        df.rename(columns={"tpep_pickup_datetime": "pickup_datetime"}, inplace=True)
        
    #deal with df.column contains list2
    elif df.columns.str.contains('pickup_datetime').any():
        df=df[['pickup_datetime','tip_amount','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']]
        #change pickup_datetime to datetime object 
        df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
        
    #deal with df.column contains list3    
    elif df.columns.str.contains('Trip_Pickup_DateTime').any():
        df=df[['Trip_Pickup_DateTime','Tip_Amt','Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat']]
        df.rename(columns={"Trip_Pickup_DateTime": "pickup_datetime",'Start_Lon':'pickup_longitude', 
                           'Start_Lat':'pickup_latitude', 'End_Lon':'dropoff_longitude', 
                           'End_Lat':'dropoff_latitude','Tip_Amt':'tip_amount'}, inplace=True)
        #change pickup_datetime to datetime object 
        df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
        
    #remove outside the latitude/longitude coordinate box
    df=df.loc[(df["pickup_longitude"]<=-73.717047) & (df["pickup_longitude"]>=-74.242330) 
    & (df["pickup_latitude"]<=40.908524) & (df["pickup_latitude"]>=40.560445)
    & (df["dropoff_longitude"]<=-73.717047) & (df["dropoff_longitude"]>=-74.242330)
    & (df["dropoff_latitude"]<=40.908524)  & (df["dropoff_latitude"]>=40.560445)]

    #remove any row with nan value
    df.dropna(inplace=True)
    #sample dataframe, since for uber dataset,roughly 2500 data per month.
    df=df.sample(n=2500,random_state = 1,ignore_index=True)
    
    return df



Function get_and_clean_taxi_data() will first find all url links. Then, each url link will be feed to function get_and_clean_month_taxi_data() to get a cleaned dataframe and add "distance" column to it. Finally, it concats all dataframes and return it as one gigantic dataframe.

In [ ]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_Parquet_urls = find_taxi_Parquet_urls()
    if os.path.exists('taxi_data.csv'):
        taxi_data=pd.read_csv('taxi_data.csv')
    else:        
        for Parquet_url in all_Parquet_urls:
            dataframe = get_and_clean_month_taxi_data(Parquet_url)
            #add distance column 
            dataframe=add_distance_column(dataframe)
            all_taxi_dataframes.append(dataframe)

        # create one gigantic dataframe with data from every month needed
        taxi_data = pd.concat(all_taxi_dataframes)
        #save file
        taxi_data.to_csv('taxi_data.csv',index=False)
    return taxi_data

### Processing Uber Data


After exploring uber dataset, we notices that we need to change the column type for "pickuo_datatime" from object to dataobject. And then selected appropriate columns for futher analysis. We also explore the Uber dataset for each month. Uber datset contains around 2500 rows for each month. Thus, we use this number to sample yellow taxi dataset, which ensure that yellow taxi dataset contain rougly equal number of sample as the uber dataset does.

Function load_and_clean_uber_data(csv_file) takes the csv file name as an input. Then it only chooses columns "pickup_datetime","pickup_longitude","pickup_latitude","dropoff_longitude", and "dropoff_latitude" from the original dataset. After that, it normalize data type of 'pickup_datetime'. Finally, it will return a cleaned Uber dataset.

In [ ]:
def load_and_clean_uber_data(csv_file):
    df=pd.read_csv(csv_file)
    #clean data
    #pick column
    df=df[["pickup_datetime","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"]]
    #column type
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime']).dt.tz_convert(None)

    return df
    

Function get_uber_data() will return a cleaned dataset with one new column "distance" (using function add_distance_column()). 

In [ ]:
def get_uber_data():
    if os.path.exists('uber_data.csv'):
        uber_dataframe=pd.read_csv('uber_data.csv')
    else:
        uber_dataframe = load_and_clean_uber_data("UBER_CSV")
        #add distance column
        uber_dataframe=add_distance_column(uber_dataframe)
        uber_dataframe=uber_dataframe.dropna()
        #store
        uber_dataframe.to_csv('uber_data.csv',index=False)
    return uber_dataframe

### Processing Weather Data

By exploratory analysis and reading the description of dataset "weather", we found only rainfall of a day exceeds a threshold that can be measured, the filed hourly precipitation does contain a numeric value. Otherwise, it will be empty or a character "T". Hence, the first step is to replace the charater "T" to empty, and then tranform data type of hourly precipitation and hourly wind speed to number. After that, we convert data type of field date to datetime and drop all the all-null row in dataframe.

Function clean_month_weather_data_hourly(csv_file) takes the csv file name as an input. Then it only chooses columns "DATE","HourlyWindSpeed", and "HourlyPrecipitation" from the original dataset. After that, it clear the unrecognized value in HourlyPrecipitation and normalize data types of 'HourlyPrecipitation', 'HourlyWindSpeed' and 'DATE'. Finally, it will return a cleaned weather dataset.

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(csv_file)
    # only select columns will be used
    selected = df[['DATE', 'HourlyWindSpeed', 'HourlyPrecipitation']]
    # convert data type to number
    selected['HourlyPrecipitation'] = pd.to_numeric(selected['HourlyPrecipitation'].replace(['T'], ''), errors='coerce')
    selected['HourlyWindSpeed'] = pd.to_numeric(selected['HourlyWindSpeed'].replace(['T'], ''), errors='coerce')
    selected['DATE'] = pd.to_datetime(selected['DATE'])
    # filter na data
    selected = selected.dropna()
    return selected

Function clean_month_weather_data_daily(csv_file) takes the csv file name as an input. Then it only chooses columns "DATE","HourlyWindSpeed", and "HourlyPrecipitation" from the original dataset. After that, it clear the unrecognized value in HourlyPrecipitation and normalize data types of 'HourlyPrecipitation', 'HourlyWindSpeed' and 'DATE'. Finally, it create two new columns with mean value of hourly wind speed and sum value of hourly percipitation, and then return a cleaned daily weather data.

In [ ]:
def clean_month_weather_data_daily(csv_file):
    df = pd.read_csv(csv_file)
    # only select columns will be used
    selected = df[['HourlyWindSpeed', 'HourlyPrecipitation', 'DATE']]
    # convert data type to number
    selected['HourlyPrecipitation'] = pd.to_numeric(selected['HourlyPrecipitation'].replace(['T'], ''), errors='coerce')
    selected['HourlyWindSpeed'] = pd.to_numeric(selected['HourlyWindSpeed'].replace(['T'], ''), errors='coerce')
    # filter na data
    selected = selected.dropna()
    # convert data time to date string
    selected['DATE'] = pd.to_datetime(selected['DATE']).dt.strftime('%Y-%m-%d')
    # get average wind speed and toal percipitation of each day
    daily_df = selected.groupby(['DATE']).agg({'HourlyWindSpeed': ['mean'], 'HourlyPrecipitation': ['sum']})
    # rename df
    daily_df.rename(columns = {'HourlyWindSpeed':'DailyWindSpeed', 'HourlyPrecipitation':'DailyPrecipitation'}, inplace = True)
    return daily_df.reset_index()

Function load_and_clean_weather_data() find all weather-like csv files in specified directory. For each weather file, it use clean_month_weather_data_hourly and clean_month_weather_data_daily functions to clean data and append cleaned reult into two lists respectively. After that, it merge and return all the cleaned dataset in each list to two dataframes.

In [ ]:
def load_and_clean_weather_data():
    import warnings
    warnings.filterwarnings('ignore')
    
    hourly_dataframes = []
    daily_dataframes = []
    
    # read all weather-like files from given path
    import glob
    weather_csv_files = list(glob.glob('./*weather*.csv'))
    weather_csv_files.sort()

    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

Call the data process function to clean taxi trip data, uber trip data, hourly weather data and daily weather data.

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data



Based on sqlalchemy, this block is designed to connect sqlite service at first and then create tables with schema sqls, which is prepared based on the data type of all dataframes we cleaned before.

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE hourly_weather(
   DATE timestamp PRIMARY KEY,
   HourlyWindSpeed FLOAT,
   HourlyPrecipitation FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE daily_weather(
   DATE timestamp PRIMARY KEY,
   DailyWindSpeed FLOAT,
   DailyPrecipitation FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

Function write_dataframes_to_table(table_to_df_dict) first convert all dataframe to sqls and then export to sqlite instance table by table.

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)